# Exploratory Data Analysis on Email Opening Behaviour


* Email Opening Prioirty of the Customer
* Data Analysed is from 1st of jan to 4th jan where there are 11Millions Mails are being sent out the customer.
* Mails can be for any reasons such as alerts,bill,account information,buy,sell,offer etc.

<b>Total Mails Sent out in this time period :11,685,864
* Mails opened by the Customers:4,414,718(37.3%)
* Mails that are not opened:7,271,148(62.3%)
</B>

<b>Next step is to perform exploratory analysis to further understand the features and relationships between various features
    
   Identify the key features that drives the customer decision to exit without completing the transaction.</B>



## Data Dictionary

There are multiple variables in the dataset which can be cleanly divided in 4 categories:

* Customer DNA Features
* Payment,Billing,Credit & Buying Features
* Calls & Transacation Features


There are many features in the data dictionary which we included in our hypothesis.


### Loading Packages
Let us load the packages needed for visualization and exploratory analysis




In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
plt.style.use("seaborn-darkgrid")
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
sns.set(style="white")

### Loading Data
Loading data from RDG as a dataframe and checking structure of the dataset

In [ ]:
query = """
select * from vzw_uda_dev_rdgtbls.email_obt_features
"""
df_ac=spark.sql(query)
#Converting to Pandas
df=df_ac.toPandas()
#Dataframe Info
df.info()

The dataset contains 19 continuous and 32 categorical variables

### Data Preprocessing Step 
* Features with a high percentage of missing values
* Collinear (highly correlated) features
* Features with zero importance in a tree-based model
* Features with low importance
* Features with a single unique value

In [ ]:
df.iloc[10,:]

We have a mix of categorical, numerical and ordinal variables as shown.

<h3> Target Exploration</h3>

We are trying to predict if the customer will Open the email sent to him and at what features that has impact over his action.

This is a binary classification problem. Let's look at the target variable and find out how many customers are in the abandon category

In [ ]:
df['om_activity_desc']=df['om_activity_desc'].fillna(0)
df['om_activity_desc']=df['om_activity_desc'].replace({"Open Email":1,
                                                                    "Clicked":1})

In [ ]:
ax = sns.catplot(y="om_activity_desc", kind="count", data=df, height=2.6, aspect=2.5, orient='h')
plt.show()

In [ ]:
df['om_activity_desc'].value_counts(normalize = True)

<b>The dataset contains 62% Not Opened  and 38% Email Opening distribution</b>

### Numerical features

Let us look at the numerical features. From the description provided in the data dictionary, we can see that we have the following numerical features. Let us quickly describe them to check the following:

* Count: Can be used to check for missing value count
* Mean: Mean of the variable
* Standard Deviation: Standard deviation of the variable
* Minimum: Minimum value of the variable
* Quantile values: 25, 50 (median) & 75% quantiles of the variable
* Maximum: Maximum value of the variable

In [ ]:
def data_distribution(data):
    """ Draws a chart showing data distribution
        by combining an histogram and a boxplot
        
    Parameters
    ----------
    data: array or series
        the data to draw the distribution for
        
    """
    
    x = np.array(data)
    
    # set the number of bins using the Rice rule
    # n_bins = twice cube root of number of observations
    n = len(x)
    n_bins = round(2 * n**(1/3))
    
    fig = plt.figure()
    
    # histogram
    ax1 = fig.add_axes([0.1, 0.3, 0.8, 0.6])
    ax1 = plt.hist(x, bins=n_bins, alpha=0.7)
    plt.grid(alpha=.5)
    
    # boxplot
    ax2 = fig.add_axes([0.1, 0.1, 0.8, 0.2])
    ax2 = plt.boxplot(x, vert=False, widths=0.7)
    plt.grid(alpha=.5)
           
    plt.show()

In [ ]:
cat_cols = ['om_activity_desc','nbs_gen','service_added','service_removed','plan_added_removed','last_payment_mode','email_status_ind','promo_removed_account','late_fee_present','port_activity','nse_ind','aal_ind','trade_in_done','mva_presence','mvo_presence','one_time_credit_ind','apo_failure','last_payment_status','eqp_pur_made']
int_cols = ['line_credit_debit_cnt','account_credit_debits_cnt','no_of_purchases','line_credits_amount','line_debits_amount','account_credits_amt','account_debits_amt','line_recur_credit_cnt','line_non_recur_credit_cnt','bill_calls','am_calls','plan_calls','payment_calls','upgrade_calls','usage_calls','troubleshooting_calls','disconnect_calls','international_calls','equipment_calls']

In [ ]:
for col in cat_cols:
    df[col]= df[col].replace('?', '0')
    df[col]= df[col].replace('#VALUE!', '0')
    df[col]= df[col].replace(' ', '0')
    df[col]= df[col].fillna(0)
    df[col]= df[col].astype('category')
    
for col in int_cols:
    df[col]= df[col].replace('?', '0')
    df[col]= df[col].replace('#VALUE!', '0')
    df[col]= df[col].replace(' ', '0')
    df[col]= df[col].fillna(0)
    df[col]= df[col].astype('int')

In [ ]:
df['calls'] = df['bill_calls'] + df['am_calls'] + df['plan_calls'] + df['payment_calls'] + df['upgrade_calls'] + df['usage_calls'] + df['troubleshooting_calls'] + df['disconnect_calls'] + df['international_calls'] + df['equipment_calls']

In [ ]:
intcons_cols = ['line_credit_debit_cnt','account_credit_debits_cnt','no_of_purchases','line_credits_amount','line_debits_amount','account_credits_amt','account_debits_amt','line_recur_credit_cnt','line_non_recur_credit_cnt','calls']

There are 19 continous and 32 categorical features available in this dataset

### Creating base line metrics

In [ ]:
df[intcons_cols].describe()

#### Lets list down a few key observations:
* On an average, a customer has made atleast a purchase in past month.
* On an average, a customer has credit of atleast 9 Dollar and atleast 4 Dollar Debit expected in his current bill 
* On an average, a customer has credit of 4 Credits from Recurrent and 0.7 from One time credit
* On an average, a customer has atleast one call to the care
* On an average, a customer has atleast one credit in the bill on the line level.
* On an average, a customer has atleast one debit in the bill on the account level.

In [ ]:
df.groupby(['om_activity_desc'])[intcons_cols].mean()

#### Customer's Email opening rate is negative when :- 
* Average credit/line is less than 11 dollars
* Average credit/line is less than 5  dollars
* Average Recurrent credit/line is less than 4.5 dollars
* Average Non Recurrent credit/line is less than 0.9 dollars
* Average credit or debit of line level is less than 8 credits.
* Average credit or debit of line level is greater than 3 credits.
* Average calls is literally none in a last days.

### Analysing the numeric feature distribution

#### Distribution of line level credit/debit

In [ ]:
data_distribution(df['line_credit_debit_cnt'])

* Line level credit/debit is positively skewed with much outliers hence it requires variable transformation.

#### Distribution of Account Credits and Debits presence

In [ ]:
data_distribution(df['account_credit_debits_cnt'])

* Account level credit/debit is positively skewed with much outliers hence it requires variable transformation.

#### Distribution of Number of Purchases

In [ ]:
data_distribution(df['no_of_purchases'])

Number of Purchases is positively skewed and hence need a log transformation

#### Total Credits per Line

In [ ]:
data_distribution(df['line_credits_amount'])

Total Credits per Line is also skewed and hence need a transformation

#### Total Debits per Line

In [ ]:
data_distribution(df['line_debits_amount'])

Total Debits is also need to be transformed as it has outliers

#### Account Credits and Debits

In [ ]:
data_distribution(df['account_credits_amt'])

In [ ]:
data_distribution(df['account_debits_amt'])

Both Account Credtis and Debits has outliers and need to be transformed

#### Number Recurrent and One time Credits

In [ ]:
data_distribution(df['line_recur_credit_cnt'])

In [ ]:
data_distribution(df['line_non_recur_credit_cnt'])

One Time Credit does not need to be transformed but Recurrent credits need to be transformed as there is an outlier

#### Distribution of calls

In [ ]:
data_distribution(df['calls'])

Calls is positively skewed with much outliers hence it requires variable transformation.

### Bivariate Analysis

Now, we will check the relationship of the numeric variables along with the target. Again conversion to log is important here as we have a lot of outliers and visualization will be difficult for it.

* <b>Log transformation for skewed variabled</b>

In [ ]:
#intcons_cols = ['line_credit_debit_cnt','account_credit_debits_cnt','no_of_purchases','line_credits_amount','line_debits_amount','account_credits_amt','account_debits_amt','line_recur_credit_cnt','line_non_recur_credit_cnt','calls']
df1 = pd.DataFrame()

for i in intcons_cols:
    df1[str('log_')+ i] = np.log(df[i] + 1)

log_cols = df1.columns

In [ ]:
#include the  var as reponse var
df1['om_activity_desc'] = df['om_activity_desc']

In [ ]:
df1_not_open = df1[df1['om_activity_desc'] == 0]
sns.pairplot(df1_not_open,vars=log_cols,plot_kws={'alpha':0.1})
plt.show()

In [ ]:
df1_open = df1[df1['om_activity_desc'] == 1]
sns.pairplot(df1_open,vars=log_cols,plot_kws={'alpha':0.1})
plt.show()

In [ ]:
sns.pairplot(df1,vars=log_cols,hue ='om_activity_desc',plot_kws={'alpha':0.1})
plt.show()

### Analysing the Numeric feature with KDE (Kernal Density Estimation) plot

* KDE Plot Smoothens out even if there are no values for a value
* The Y axis gives the probablity density function (AUC)

In [ ]:
# KDE Plot Smoothens out even if there are no values for a value
# The Y axis gives the probablity density function (AUC)
def kdeplot(feature):
    plt.figure(figsize=(9, 4))
    plt.title("KDE Plot for {}".format(feature))
    ax0 = sns.kdeplot(df[df['om_activity_desc'] == 0][feature].dropna(), color= 'dodgerblue', label= 'Not Opened - 0')
    ax1 = sns.kdeplot(df[df['om_activity_desc'] == 1][feature].dropna(), color= 'orange', label= 'Opened - 1')

kdeplot('line_credit_debit_cnt')
plt.show()

* The plot clearly shows that if the customer has more than 3.5 credit/debit per line then chance of opening mail is high.

In [ ]:
kdeplot('account_credit_debits_cnt')
plt.show()

* The plot does not clearly diffenciate between the opening and not opening.

In [ ]:
kdeplot('line_credits_amount')
plt.show()

The Graph clearly shows as there is more credit more chances of opening mail

In [ ]:
kdeplot('line_recur_credit_cnt')
plt.show()

The Graph clearly shows person with more than 8 recurrent credits to his is opening the mail more often

In [ ]:
kdeplot('no_of_purchases')
plt.show()

Purchases contribute to the mailing activity as it is shown by the graph.i.e persons with more than 1 purchases in a month has the tendency to open the mail

### Categorical features

This dataset has  30 + categorical features as can be inferred from the data dictionary. Now let us have a look at the the number of unique values for each of them.

In [ ]:
for i in range(0,len(cat_cols)):
    print(str(cat_cols[i]) + " - Number of Unique Values: " + str(df[cat_cols[i]].nunique()))

* As there are no features which has more levels, these categorical variables doesnot require transformation expect few.

### Bivariate Analysis

Lets define a function to quickly compare churn rates for different categories in each feature

In [ ]:
def barplot_percentages(feature):
    #fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(16, 6))
    #fig, ax = plt.subplots()
    ax1 = df.groupby(feature)['om_activity_desc'].value_counts(normalize=True).unstack()
    #plt.figure(figsize=(2,3))
    ax1.plot(kind='bar', stacked='True',figsize=(8,4))
    int_level = df[feature].value_counts()

    plt.figure(figsize=(8,4))
    sns.barplot(int_level.index, int_level.values, alpha=0.8)
    plt.ylabel('Number of Occurrences', fontsize=12)
    plt.xlabel(str(feature), fontsize=12)
    plt.show()

#### Email Subject/Email Topics
<b>This Topic is generated by using the NLP Topic Generator prior to the preprocessing.</b>


In [ ]:
barplot_percentages("Topics")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "Topics", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['Topics'])

The Topics is one of the major factor in the Mail opening.In this Data it can be defined by the number of opening and by the proportion of the opening.

<b>Top Topics based on the numbers </b>                          
* Alerts                                                         
* Account Maintiance                                                               
* Bill & Credits                                                 
* Payment & Collections                                         
* Order                                                          

<b>Top Topics based on the proportion </b>
* Technical & Support
* Order                    
* Promotion & Discounts
* Port
* Equipment

<b> These Topics contributes to about 76% of the mails that are being opened.</b>

#### NBS Generator

In [ ]:
barplot_percentages("nbs_gen")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "nbs_gen", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['nbs_gen'])

* As expected the nbs genaration does not make much of the difference between mail opening.The customer would be seeing the nbs in his profile itslef after genaration request
But People with the NBS tend to open the mail <b> 15.19% which is 3% more than the average opening range of 12.66% <b>

#### Service Added

In [ ]:
barplot_percentages("service_added")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "service_added", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['service_added'])

* Service added has significant difference between opening the mail and not opening as people who added the service open the mail in the range of <b> 39.13% </b> as compared to not opening % of <b> 34.93 </b>.

#### Service Removed

In [ ]:
barplot_percentages("service_removed")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "service_removed", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['service_removed'])

Service Removed has an added <b>2%(19.31%)</b> advantage on the mail opening as compared to an average of 17.49%

#### Plan Change (Add/Remove)

In [ ]:
barplot_percentages("plan_added_removed")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "plan_added_removed", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['plan_added_removed'])

Plan Change has a non opening % of 44.73 and opening % of <b>49.56</b> which is an <b>5%</b> difference contributing to the mail opening

#### Last Payment Mode

In [ ]:
barplot_percentages("last_payment_mode")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "last_payment_mode", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['last_payment_mode'])

Customer who had opened mail are mostly among the last payment modes of <b>AH,CR,DD,WC</b>

#### Last Email  Opening status

In [ ]:
barplot_percentages("email_status_ind")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "email_status_ind", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['email_status_ind'])

#### Promotion Changes

In [ ]:
barplot_percentages("promo_removed_account")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "promo_removed_account", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['promo_removed_account'])

* <b>Surprisingly Promotional Changes does not have any impact on the mail opening (opening-3%,not opening - 2.67%)</b>

#### Late Fee 

In [ ]:
barplot_percentages("late_fee_present")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "late_fee_present", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['late_fee_present'])

* <b>Again Surprisingly People who had late fee open mail only  in the range of 35.05% as compared to 49.05% who does not open the mail</b>

#### Port Activity

In [ ]:
barplot_percentages("port_activity")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "port_activity", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['port_activity'])

<b>Customers who has done a porting realted activity are more likely to open the mail clearly</b>

#### NSE and AAL

In [ ]:
barplot_percentages("nse_ind")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "nse_ind", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['nse_ind'])

In [ ]:
barplot_percentages("aal_ind")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "aal_ind", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['aal_ind'])

AAL & NSE Contributes to an added advantage of <b>2%</b> contribution towards the mail opening to the average rate of <b>3.2%</b>

#### Trade In Done

In [ ]:
barplot_percentages("trade_in_done")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "trade_in_done", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['trade_in_done'])

Trade In Context has very less impact on the mail opening

#### Digital Presence (MVO & MVA)

In [ ]:
barplot_percentages("mva_presence")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "mva_presence", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['mva_presence'])

In [ ]:
barplot_percentages("mvo_presence")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "mvo_presence", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['mvo_presence'])

* One of the most significant factor is this Digital visits,<b>86.6%</b> Customers who had opened mail had atleast onw digital touchpoint in the past week as compared to the <b>75.4%</b> of the customers who had a touchpoint but has not opened the mail.
* This <b>11%</b> increase is one of the significant factor for the mail opening factor

#### Last payment Status

In [ ]:
barplot_percentages("last_payment_status")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "last_payment_status", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['last_payment_status'])

#### One Time Credit/Credit starting from current month

In [ ]:
barplot_percentages("one_time_credit_ind")
#Relationship between two categori

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "one_time_credit_ind", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['one_time_credit_ind'])

* Credtis have an upper hand in this though very little.But as per % probability of opening mail <b>59%</b> when ever the customer has a credit even though numbers are small

#### APO Failure

In [ ]:
barplot_percentages("apo_failure")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "apo_failure", data = df)


In [ ]:
pd.crosstab(df['om_activity_desc'], df['apo_failure'])

* APO Failure does not contribute the mail opening as this failure might be informed to other modes of notifications and hence the need to open this mail is very minimal

#### Equipment Purchase

In [ ]:
barplot_percentages("eqp_pur_made")

In [ ]:
#Relationship between two categorical variables
sns.countplot(x="om_activity_desc", hue = "eqp_pur_made", data = df)

In [ ]:
pd.crosstab(df['om_activity_desc'], df['eqp_pur_made'])

* People who bought equipment/Product in last month has <b>28.8% opened mail which is 2% more than that of the average opening rate of 26.37</b>

## Conclusion

#### Average customer Profile
Overall a customer:
* has made atleast a purchase in past month.
* has credit of atleast 9 Dollar and atleast 4 Dollar Debit expected in his current bill 
* has credit of 4 Credits from Recurrent and 0.7 from One time credit
* has atleast one call to the care
* has atleast one credit in the bill on the line level.
* has atleast one debit in the bill on the account level.


#### Conclusion for Emails Opened
* From the sample, around 37.7% mails are being opened. 
* Numeric features values have a right skewed distribution as observed from the histogram.
* Email Topic has huge impact on opening rate.
* Higher the credit amount in the current bill cycle higher the opening rate.
* If the customer has more than 8 credtis, it increases the opening rate.
* Customer who Generated NBS and had an APO Failure has low opening rate.
* Customer who purchased something in the past , chance of opening is high.
* AAL,NSE has an positive impact on the opening of mail.
* Payment Type of the last payment and last payment status contributes to the mail opening.